# Day 6: Universal Orbit Map
https://adventofcode.com/2019/day/6

## Part 1

In [ ]:
import numpy as np
import urllib.request
import math

In [ ]:
class OMNode():
    key = None
    parentKey = None
    
    def __init__(self, key):
        self.key = key
    
    def toString(self):
        return 'NODE {} ORBITS {}'.format(self.key, self.parentKey)


class OMMap():
    nodes = None
    nodeOrigen = None
    
    def __init__(self, originNode):
        self.nodes = {}
        self.nodeOrigen = originNode
    
    def loadMap(self, rawMap):
        rawMap = rawMap.split('\n')
        
        # Adding origin node to map
        originNode = OMNode(self.nodeOrigen)
        self.nodes[originNode.key] = originNode
        
        for orbit in rawMap:
            if len(orbit) == 0:
                continue
            items = orbit.split(')')
#             print(items)
            assert( len(items) == 2 )
            parentKey = items[0]
            childKey = items[1]
            
#             print('{} ORBITS {}'.format(childKey, parentKey))
            if parentKey not in self.nodes:
                parentNode = OMNode(parentKey)
                self.nodes[parentNode.key] = parentNode
            if childKey not in self.nodes:
                childNode = OMNode(childKey)
                self.nodes[childNode.key] = childNode
            # Assing parent to child
            # TODO: CHECK IF CHILD ALREADY HAS ANOTHER PARENT!!
            childNode = self.nodes[childKey]
            childNode.parentKey = parentKey
            
    def printMap(self):
        for i in self.nodes:
            print(self.nodes[i].toString())
    
    def getDirectOrbitFrom(self, nodeKey):
        if self.nodes[nodeKey].parentKey in self.nodes:
            return self.nodes[self.nodes[nodeKey].parentKey]
        else:
            return None

    def getIndirectOrbitsFrom(self, nodeKey):
        indirects = []
        maxlimit = 5000
        limit = 0
        
        node = self.nodes[nodeKey]
        
        if node.parentKey == None:
            return indirects
        
        nodeKey = node.parentKey
        while True:
            limit += 1
            if limit > maxlimit:
                raise Exception('Iteration limit {} exceded!!!'.format(maxlimit))

            parent = self.getDirectOrbitFrom(nodeKey)
            if parent == None:
                break
            else:
                indirects.append(parent)
                nodeKey = parent.key
                
        return indirects

    def getTotalOrbits(self):
        totalOrbits = 0
        for node in self.nodes:
            if self.getDirectOrbitFrom(node) != None:
                totalOrbits += 1
            totalOrbits += len(self.getIndirectOrbitsFrom(node))
        return totalOrbits

### Tests

Given this input:

<code>
COM)B
B)C
C)D
D)E
E)F
B)G
G)H
D)I
E)J
J)K
K)L
</code>

<code>
        G - H       J - K - L
       /           /
COM - B - C - D - E - F
               \
                I
</code>

In [ ]:
rawMap = '''COM)B
B)C
C)D
D)E
E)F
B)G
G)H
D)I
E)J
J)K
K)L'''

print(rawMap)

In [ ]:
def testMap(testNode):
    directOrbit = orbitMap.getDirectOrbitFrom(testNode)
    print('Check direct orbit:')
    orbitsSoFar = 0
    if directOrbit == None:
        print('Node {} has no orbits'.format(testNode))
    else:
        print('Node {} orbits {}'.format(testNode, directOrbit.key))
        orbitsSoFar = 1
    
    print('Check indirect orbit:')
    indirects = orbitMap.getIndirectOrbitsFrom(testNode)
    if indirects == None or len(indirects) == 0:
        print('Node {} has no indirect orbits'.format(testNode))
    else:
        for node in indirects:
            print(node.toString())
        orbitsSoFar += len(indirects)
    print('Node {} has {} total orbits'.format(testNode, orbitsSoFar))

In [ ]:
orbitMap = OMMap('COM')
orbitMap.loadMap(rawMap)
orbitMap.printMap()

#### Test 1
* D directly orbits C and indirectly orbits B and COM, a total of 3 orbits.

In [ ]:
testMap('D')

#### Test 2
* L directly orbits K and indirectly orbits J, E, D, C, B, and COM, a total of 7 orbits.

In [ ]:
testMap('L')

#### Test 3
* COM orbits nothing.

In [ ]:
testMap('COM')

#### Test 4
* The total number of direct and indirect orbits in this example is 42.

In [ ]:
orbitMap.getTotalOrbits()

### Solution

In [ ]:
input_6 = r'data\aoc2019-input-day6.txt'
with open(input_6, 'r') as f:
#     data5 = [int(data) for data in f.read().split(',') if len(data) > 0]
    data6 = f.read()
print(data6)

In [ ]:
orbitMap = OMMap('COM')
orbitMap.loadMap(data6)


In [ ]:
orbitMap.getTotalOrbits()

Total orbits: 314702

## Part 2

In [ ]:
def solveDay6Part2(rawMap, fromNode, toNode, debug = False):
    orbitMap = OMMap('COM')
    orbitMap.loadMap(rawMap)
    if debug:
        orbitMap.printMap()
    
    #1. Mis órbitas
    youOrbits = orbitMap.getIndirectOrbitsFrom(fromNode)
    youOrbits.reverse()
    youOrbits = youOrbits + [orbitMap.getDirectOrbitFrom(fromNode)]
    
    sanOrbits = orbitMap.getIndirectOrbitsFrom(toNode)
    sanOrbits.reverse()
    sanOrbits = sanOrbits + [orbitMap.getDirectOrbitFrom(toNode)]
    
    youName = orbitMap.nodes[fromNode].key
    sanName = orbitMap.nodes[toNode].key
    
    if debug:
        print('{} orbits:'.format(youName))
        for node in youOrbits:
            print(node.key)
        print('{} orbits:'.format(sanName))
        for node in sanOrbits:
            print(node.key)
    
    while True:
        nodeYOU = youOrbits[0]
        if sanOrbits[0] == nodeYOU:
            youOrbits = youOrbits[1:]
            sanOrbits = sanOrbits[1:]
        else:
            break
    
    nexus = orbitMap.nodes[youOrbits[0].parentKey]
    if debug:
        print('NEXUS:', nexus.key)

    if debug:
        print('After the cull:')
        print('{} orbits:'.format(youName))
        for node in youOrbits:
            print(node.key)
        print('{} orbits:'.format(sanName))
        for node in sanOrbits:
            print(node.key)

    movements = []
    youOrbits.reverse()
    if debug:
        print('{} orbits:'.format(youName))
        for node in youOrbits:
            print(node.key)
    
    # Traverse my orbits :
    for i in range(len(youOrbits)):
        if i > 0:
            movements.append(( youOrbits[i-1].key, youOrbits[i].key ))
    
    movements.append( (youOrbits[len(youOrbits) -1].key , nexus.key) )
    movements.append( (nexus.key, sanOrbits[0].key))
    
    for i in range(len(sanOrbits)):
        if i > 0:
            movements.append( ( sanOrbits[i-1].key, sanOrbits[i].key))
    
    return movements

In [ ]:
rawMap = '''COM)B
B)C
C)D
D)E
E)F
B)G
G)H
D)I
E)J
J)K
K)L
K)YOU
I)SAN'''

print(rawMap)

### Tests

Given this input:

<code>
COM)B
B)C
C)D
D)E
E)F
B)G
G)H
D)I
E)J
J)K
K)L
K)YOU
I)SAN
</code>

<code>
                          YOU
                         /
        G - H       J - K - L
       /           /
COM - B - C - D - E - F
               \
                I - SAN
</code>

In this example, YOU are in orbit around K, and SAN is in orbit around I. To move from K to I, a minimum of 4 orbital transfers are required:

* K to J
* J to E
* E to D
* D to I

In [ ]:
movements = solveDay6Part2(rawMap, 'YOU', 'SAN')
print(movements)
print(len(movements))

### Solution

In [ ]:
movements = solveDay6Part2(data6, 'YOU', 'SAN')
print(movements)
print(len(movements))

439 movements